In [1]:
import requests
from bs4 import BeautifulSoup

In [2]:
def get_doc():
    url = 'https://www.imdb.com/list/ls057577566/?sort=moviemeter,asc&st_dt=&mode=detail&page=1'
    response = requests.get(url)
    if response.status_code != 200:
        raise Exception("page couldn't be open")
    soup = BeautifulSoup(response.text, 'html.parser')
    return soup

In [3]:
soup = get_doc()

In [4]:
list_names = []
list_ratings = []
list_genres = []
list_descriptions = []

In [5]:
def get_data(soup):
    name = soup.select('div.lister-item-content > h3.lister-item-header > a[href^="/title/"]')

    for a in name:
        list_names.append(a.text)
    
    rating_parent_div_tags = soup.select('div.ipl-rating-widget')
    
    for parent_div_tag in rating_parent_div_tags:
        div_tag = parent_div_tag.find('div', {'class': 'ipl-rating-star small'})
        span_tag = div_tag.find('span', {'class': 'ipl-rating-star__rating'})
        list_ratings.append(span_tag.text)

    genre = soup.select('span.genre')
    for g in genre:
        list_genres.append(g.text.strip().replace('Animation, ', ''))
        
    description_tags = soup.select('div.lister-item-content')
    for tag in description_tags:
        description = tag.find('p', {'class': ''}).text
        list_descriptions.append(description)

In [6]:
get_data(soup)

In [7]:
for i, desc in enumerate(list_descriptions):
    replaced = desc.replace('\n', '')
    list_descriptions[i] = replaced.replace('                See full summary\xa0»', '')

In [8]:
dict_of_lists = {
    'Name'      : list_names,
    'Ratings'   : list_ratings,
    'Genres'    : list_genres,
    'Description': list_descriptions
}

In [9]:
import pandas as pd
anime_df = pd.DataFrame(dict_of_lists)

In [10]:
anime_df

,Name,Ratings,Genres,Description
0,Demon Slayer: Kimetsu No Yaiba,8.7,"Action, Adventure",A family is attacked by demons and only two me...
1,Hunter x Hunter,9,"Action, Adventure","Gon Freecss aspires to become a Hunter, an exc..."
2,Death Note: Desu nôto,9,"Crime, Drama",An intelligent high school student goes on a s...
3,Vinland Saga,8.8,"Action, Adventure",Thorfinn pursues a journey with his father's k...
4,JoJo no kimyô-na bôken,8.4,"Action, Adventure","The story of the Joestar family, who are posse..."
...,...,...,...,...
95,Koi kaze,7.5,"Drama, Romance","Koshiro and Nanoka fall deeply in love, then d..."
96,Ef: A Tale of Memories.,7.5,"Comedy, Drama","A story of love, heartbreak and youth, as the ..."
97,Sayonara zetsubô sensei,7.6,"Comedy, Drama",A pessimistic high school teacher must somehow...
98,Kaleido Star,7.8,"Comedy, Drama","Sora, a young girl from Japan, comes to Americ..."


In [11]:
anime_df.to_csv('list-popular-anime.csv')